In [1]:
#!pip install web3

In [2]:
import pandas as pd
import requests
import datetime
import time
import json
from web3 import Web3 as web3
import numpy as np
from web3 import Web3

In [ ]:
# Initialize a Web3 instance
infura_project_id = 'YOUR-PROJECT-ID'
infura_url = f'https://mainnet.infura.io/v3/{infura_project_id}'

web3 = Web3(Web3.HTTPProvider(infura_url))

In [28]:
#introduce a caching layer to reduce API calls

from web3.middleware import geth_poa_middleware, simple_cache_middleware
# add the geth_poa_middleware to handle POA chains
web3.middleware_onion.inject(geth_poa_middleware, layer=0)

# add the simple_cache_middleware to cache responses
web3.middleware_onion.add(simple_cache_middleware)

In [ ]:
#load top token addresses and block numbers that will be needed later on
top_tokens_addresses = pd.read_csv('../../utils/price-data/top_tokens_addresses.csv') 
block_numbers_df = pd.read_csv('../../utils/block-numbers/block_numbers_2020-01-2023-04.csv')

<h1 style="color: blue; font-size: 36px; font-family: Arial;">DyDX L2PerpetualContract Escrow</h1>

In [ ]:
#contract-address: 0xD54f502e184B6B739d7D27a6410a67dc462D69c8
#created at Feb-11-2021  #at block 11834295

In [6]:
import pandas as pd
import requests

# Replace API_KEY with your own Ethplorer API key
API_KEY = "YOUR-API-KEY"

# Replace CONTRACT_ADDRESS with the address of the contract you want to query
CONTRACT_ADDRESS = "0xD54f502e184B6B739d7D27a6410a67dc462D69c8"

# Construct the API endpoint URL
url = f"https://api.ethplorer.io/getAddressInfo/{CONTRACT_ADDRESS}?apiKey={API_KEY}"

# Send the API request and parse the response as JSON
response = requests.get(url)
data = response.json()
# Get the list of token balances held by the contract
token_balances = data.get("tokens", [])

# Create an empty DataFrame
token_addresses_held_dydx_escrow = pd.DataFrame(columns=["Token Address"])

# Append each token address to a new row in the DataFrame
for token_balance in token_balances:
    token_address = token_balance.get("tokenInfo", {}).get("address")
    token_addresses_held_dydx_escrow = token_addresses_held_dydx_escrow.append({"Token Address": token_address}, ignore_index=True)

# Print the DataFrame
print(token_addresses_held_dydx_escrow)

                                 Token Address
0   0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48
1   0x92d6c1e31e14520e676a687f0a93788b716beff5
2   0x582685797a7f2faca82d49c6c7d166edc7ed3bfb
3   0x1e0fad3fc0fd2d1479bc5a5e09c6b88a3426fd7f
4   0x17a10104cbc1ed155d083ead9fcf5c3440bb50e8
5   0x1f068a896560632a4d2e05044bd7f03834f1a465
6   0x12e951934246186f50146235d541d3bd1d463e4d
7   0x956f824b5a37673c6fc4a6904186cb3ba499349b
8   0x68ca006db91312cd60a2238ce775be5f9f738bba
9   0x643695d282f6ba237afe27ffe0acd89a86b50d3e
10  0xa6de609807c7258a0d34f5307c1808f062a59794
11  0x53fffb19bacd44b82e204d036d579e86097e5d09


In [9]:
# all token_addresses held by the escrow that also appear in the top 300 of CoinGecko
dydxEscrow_top_token_addresses = pd.merge(top_tokens_addresses, token_addresses_held_dydx_escrow, left_on="token_address", right_on="Token Address")
dydxEscrow_top_token_addresses['token_address'] = dydxEscrow_top_token_addresses['token_address'].apply(lambda x: web3.to_checksum_address(x))


Token balance for address 0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48 at for 2021-02-11 is: 0
Token balance for address 0x92d6c1e31e14520e676a687f0a93788b716beff5 at for 2021-02-11 is: 0
Token balance for address 0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48 at for 2021-02-12 is: 0
Token balance for address 0x92d6c1e31e14520e676a687f0a93788b716beff5 at for 2021-02-12 is: 0
Token balance for address 0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48 at for 2021-02-13 is: 0
Token balance for address 0x92d6c1e31e14520e676a687f0a93788b716beff5 at for 2021-02-13 is: 0
Token balance for address 0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48 at for 2021-02-14 is: 0
Token balance for address 0x92d6c1e31e14520e676a687f0a93788b716beff5 at for 2021-02-14 is: 0
Token balance for address 0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48 at for 2021-02-15 is: 0
Token balance for address 0x92d6c1e31e14520e676a687f0a93788b716beff5 at for 2021-02-15 is: 0
Token balance for address 0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48 a

In [30]:
#dydx_escrow_token_balances_per_day.to_csv('L2PerpetualContract/dydx_escrow_token_balances_per_day.csv', index=False)


In [8]:
dydx_escrow_balances = pd.read_csv('L2PerpetualContract/dydx_escrow_token_balances_per_day.csv') 
dydx_escrow_balances['token_address'] = dydx_escrow_balances['token_address'].apply(lambda x: web3.to_checksum_address(x) if x != 'ether' else x)
dydx_escrow_balances['date'] = pd.to_datetime(dydx_escrow_balances['date'])
dydx_escrow_balances = dydx_escrow_balances[dydx_escrow_balances['date'] < '2023-04-30']
dydx_escrow_balances['token_address'] = dydx_escrow_balances['token_address'].apply(str)
dydx_escrow_balances = dydx_escrow_balances.drop(columns=['has_error'])

In [ ]:
#date of the contract deployment
start_date = '2021-02-11'

# Get the index position of the start date in the DataFrame
start_index = block_numbers_df[block_numbers_df['Date'] == start_date].index[0]

# Slice the DataFrame from the start index onwards
block_numbers_slice = block_numbers_df.iloc[start_index:]

#new fin try
def get_token_balance(token_holding_address_checksum, token_address, contract, block_number):
    try:
        token_balance = contract.functions.balanceOf(token_holding_address_checksum).call(block_identifier=block_number)
        has_error = False
    except Exception as e:
        token_balance = 0
        has_error = True
        #print(f"Error getting balance for {token_address}: {e}")
    return token_balance, has_error

# Define the ERC20 contract ABI
abi = [
    {
        'inputs': [],
        'name': 'decimals',
        'outputs': [{'internalType': 'uint8', 'name': '', 'type': 'uint8'}],
        'stateMutability': 'view',
        'type': 'function'
    },
    {
        'inputs': [{'internalType': 'address', 'name': '', 'type': 'address'}],
        'name': 'balanceOf',
        'outputs': [{'internalType': 'uint256', 'name': '', 'type': 'uint256'}],
        'stateMutability': 'view',
        'type': 'function'
    }
]

# Define the token addresses DataFrame
token_addresses_df = dydxEscrow_top_token_addresses

token_holding_address = "0xD54f502e184B6B739d7D27a6410a67dc462D69c8"
token_holding_address_checksum = web3.to_checksum_address(token_holding_address)

# Create a contract instance for each token address
contracts = {}
for _, row in token_addresses_df.iterrows():
    token_address = row['Token Address']
    token_address_checksum = web3.to_checksum_address(token_address)
    contract = web3.eth.contract(address=token_address_checksum, abi=abi)
    contracts[token_address] = contract

# Retrieve the token balances for each token address and block number, and store the results in a DataFrame
balances = []
for i, row in block_numbers_slice.iterrows():
    #print(f"Retrieving token balances for {row['Date']} - Block Number:{row['Block Number']} ")
    for _, row2 in token_addresses_df.iterrows(): 
        token_decimals = row2['decimals']
        token_address = row2['Token Address']
        block_number = row['Block Number']
        contract = contracts[token_address]
        token_balance, has_error = get_token_balance(token_holding_address_checksum, token_address, contract, block_number)
        #print(f"On day {row['Date']} token address {token_address} had a balance of {token_balance} ")
        print(f"Token balance for address {token_address} at for {row['Date']} is: {token_balance}")
        time.sleep(0.5)
        balances.append({
            'date': row['Date'],
            'block_number': block_number,
            'token_address': token_address,
            'token_name': row2['coingecko_id'],
            'token_decimals': row2['decimals'],
            'balance': token_balance / 10**token_decimals,
            'has_error': has_error
        })

dydx_escrow_token_balances_per_day = pd.DataFrame(balances)


In [9]:
price_data = pd.read_csv('../../utils/price-data/price_data.csv') 
price_data['token_address'] = price_data['token_address'].apply(lambda x: web3.to_checksum_address(x) if x != 'ether' else x)
price_data['timestamp'] = pd.to_datetime(price_data['timestamp'])
price_data['token_address'] = price_data['token_address'].apply(str)

In [10]:
merged_df = dydx_escrow_balances.merge(price_data, left_on=['token_address', 'date'], right_on=['token_address', 'timestamp'], how='left')


In [ ]:
#some missing rows due to coingecko not listing the price right after deployment of the token address
missing_rows = merged_df.loc[merged_df['timestamp'].isna() | merged_df['price'].isna()]
missing_rows

In [12]:
dydx_escrows_final = dydx_escrow_balances.merge(price_data, left_on=['token_address', 'date'], right_on=['token_address', 'timestamp'])

In [13]:
# Add a new column with the desired constant value
dydx_escrows_final['escrow_address'] = '0xD54f502e184B6B739d7D27a6410a67dc462D69c8'
dydx_escrows_final['escrow_address_rdbl'] = 'L2PerpetualContract'
dydx_escrows_final['date'] = pd.to_datetime(dydx_escrows_final['date'])

In [10]:
dydx_escrows_final['balance_usd'] = dydx_escrows_final['balance'] * dydx_escrows_final['price']


In [18]:
#can be disregarded and will be overwritten with the correct value in eth in the analysis part
dydx_escrows_final['token_amount'] = dydx_escrows_final['balance'] * 10 ** dydx_escrows_final['token_decimals']

def wei_to_eth(wei_value):
    # 1 Ether (ETH) is equivalent to 10^18 Wei
    eth_value = wei_value * 1e-18
    return eth_value

# Apply the conversion function and add a new column
dydx_escrows_final["balance_eth"] = dydx_escrows_final["token_amount"].apply(wei_to_eth)

### save final df

In [15]:
#dydx_escrows_final.to_csv('../../data/dydx_escrows_final.csv', index=False)